# Physiognomy Analysis API Test

This notebook demonstrates how to test the Physiognomy Analysis API. 

**Current API Format**: The API returns JSON with URLs to access images separately
**Alternative**: We'll also show how to modify the API to return a ZIP file

## Option 1: Testing Current JSON API Format

In [ ]:
# Import Required Libraries
import requests
import zipfile
import io
import json
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import base64

In [ ]:
# Configuration
API_URL = "https://physiognomy-xg3f.onrender.com/analyze/"

# Update this path to your test image
IMAGE_PATH = "test_image.jpg"  # Change this to your actual image path

print(f"🔧 API URL: {API_URL}")
print(f"🖼️ Image Path: {IMAGE_PATH}")

In [ ]:
# Test Current JSON API Format
print(f"🚀 Sending image '{IMAGE_PATH}' to server...")

try:
    # Check if image file exists
    if not Path(IMAGE_PATH).exists():
        print(f"❌ Error: Image file '{IMAGE_PATH}' not found!")
        print("Please upload an image file or update IMAGE_PATH variable")
    else:
        with open(IMAGE_PATH, 'rb') as image_file:
            files = {'file': (IMAGE_PATH, image_file, 'image/jpeg')}
            
            # Send request to API
            response = requests.post(API_URL, files=files, timeout=120)
            
            print(f"📡 Response Status Code: {response.status_code}")
            
            if response.status_code == 200:
                print("✅ Request successful!")
                
                # Parse JSON response
                try:
                    result_data = response.json()
                    print("\n📊 Analysis Results:")
                    print(json.dumps(result_data, indent=2, ensure_ascii=False))
                    
                    # Check if visuals URLs are available
                    if "visuals" in result_data:
                        print(f"\n🖼️ Annotated Image URL: {result_data['visuals']['annotated_image_url']}")
                        print(f"📋 Report Image URL: {result_data['visuals']['report_image_url']}")
                        
                        # Store URLs for next cell
                        annotated_url = result_data['visuals']['annotated_image_url']
                        report_url = result_data['visuals']['report_image_url']
                    else:
                        print("⚠️ No visual URLs found in response")
                        
                except json.JSONDecodeError:
                    print("❌ Failed to parse JSON response")
                    print("Raw response content:")
                    print(response.text[:500])  # First 500 chars
                    
            else:
                print(f"❌ Request failed! Status Code: {response.status_code}")
                print("Error details:")
                try:
                    error_data = response.json()
                    print(json.dumps(error_data, indent=2))
                except:
                    print(response.text)
                    
except FileNotFoundError:
    print(f"❌ Error: Could not find image file at '{IMAGE_PATH}'")
    print("Please make sure the file exists or update the IMAGE_PATH variable")
except requests.exceptions.RequestException as e:
    print(f"❌ Connection error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

In [ ]:
# Download and Display Images from URLs
# (Run this cell only if the previous cell was successful and URLs were obtained)

try:
    if 'annotated_url' in locals() and 'report_url' in locals():
        print("🖼️ Downloading and displaying images...")
        
        # Download annotated image
        print("📥 Downloading annotated image...")
        annotated_response = requests.get(annotated_url, timeout=30)
        if annotated_response.status_code == 200:
            annotated_image = Image.open(io.BytesIO(annotated_response.content))
            
            # Display annotated image
            plt.figure(figsize=(10, 8))
            plt.imshow(annotated_image)
            plt.title("Annotated Image", fontsize=16)
            plt.axis('off')
            plt.show()
        else:
            print(f"❌ Failed to download annotated image: {annotated_response.status_code}")
        
        # Download report image
        print("📥 Downloading report image...")
        report_response = requests.get(report_url, timeout=30)
        if report_response.status_code == 200:
            report_image = Image.open(io.BytesIO(report_response.content))
            
            # Display report image
            plt.figure(figsize=(10, 15))
            plt.imshow(report_image)
            plt.title("Analysis Report", fontsize=16)
            plt.axis('off')
            plt.show()
        else:
            print(f"❌ Failed to download report image: {report_response.status_code}")
            
    else:
        print("⚠️ No image URLs available. Please run the previous cell successfully first.")
        
except Exception as e:
    print(f"❌ Error downloading images: {e}")

## Option 2: Alternative ZIP File API (Requires API Modification)

The code below shows how to handle a ZIP file response if you modify your API to return a ZIP file instead of JSON with URLs.

**To implement this, you would need to add a new endpoint to your API that returns a ZIP file containing:**
- `annotated_image.jpg`
- `report.jpg` 
- `analysis.json`

In [ ]:
# Original ZIP File Handling Code (for reference)
# This is the code you were trying to use - it expects a ZIP file response

def test_zip_api(api_url, image_path):
    """Test function for ZIP file API response"""
    print(f"🚀 Sending image '{image_path}' to server...")
    
    try:
        with open(image_path, 'rb') as image_file:
            files = {'file': (image_path, image_file, 'image/jpeg')}
            response = requests.post(api_url, files=files, timeout=90)

        if response.status_code == 200:
            print("✅ Request successful! Processing received ZIP file...")

            zip_buffer = io.BytesIO(response.content)

            # Open ZIP file from buffer
            with zipfile.ZipFile(zip_buffer, 'r') as zip_ref:
                print("📁 ZIP file contents:", zip_ref.namelist())

                # 1. Read and display annotated image
                if 'annotated_image.jpg' in zip_ref.namelist():
                    with zip_ref.open('annotated_image.jpg') as annotated_file:
                        annotated_image = Image.open(annotated_file)
                        plt.figure(figsize=(8, 8))
                        plt.imshow(annotated_image)
                        plt.title("Annotated Image")
                        plt.axis('off')
                        plt.show()

                # 2. Read and display report image
                if 'report.jpg' in zip_ref.namelist():
                    with zip_ref.open('report.jpg') as report_file:
                        report_image = Image.open(report_file)
                        plt.figure(figsize=(10, 15))
                        plt.imshow(report_image)
                        plt.title("Analysis Report")
                        plt.axis('off')
                        plt.show()

                # 3. Read and display JSON data
                if 'analysis.json' in zip_ref.namelist():
                    with zip_ref.open('analysis.json') as json_file:
                        analysis_data = json.loads(json_file.read().decode('utf-8'))
                        print("\n--- 📊 Analysis JSON Data ---")
                        print(json.dumps(analysis_data, indent=4, ensure_ascii=False))

        else:
            print(f"❌ Request failed! Status Code: {response.status_code}")
            print("Error details:")
            print(response.text)

    except FileNotFoundError:
        print(f"❌ Error: Could not find image file at '{image_path}'")
    except requests.exceptions.RequestException as e:
        print(f"❌ Connection error: {e}")
    except zipfile.BadZipFile:
        print("❌ Error: Response is not a valid ZIP file")
        print("Response content type:", response.headers.get('content-type'))
        print("First 100 chars of response:", response.text[:100])
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

# Uncomment the line below to test with ZIP API (if you implement it)
# test_zip_api(API_URL, IMAGE_PATH)

## Troubleshooting

### Common Issues and Solutions:

1. **"Response ended prematurely"**: 
   - This usually means the server closed the connection unexpectedly
   - Check if the server is running and accessible
   - Try with a smaller image file
   - Increase timeout value

2. **File not found**: 
   - Make sure your image file exists at the specified path
   - Update the `IMAGE_PATH` variable to the correct location

3. **API returning JSON instead of ZIP**:
   - Your current API returns JSON with image URLs (which is working correctly)
   - The ZIP file code was expecting a different API format

4. **Server errors (500)**:
   - Check server logs for detailed error information
   - Ensure all required models are loaded properly
   - Verify image format is supported (JPG/PNG)